# Chapter 16 코루틴
- `yield` 키워드 뒤에 표현식이 없으면 `None`을 생성
- `next()` 대신 `send()` 메서드를 호출하면 코루틴이 호출자로부터 데이터를 받을 수 있다.
  - `send()`를 통해 호출자가 코루틴에 값을 밀어 넣는다.
  - 물론, 아무 데이터를 주고 받지 않을 수도 있다. 이 경우 `yield`는 실행을 제어하는 장치로서 멀티태스킹에서의 협업을 구현하기 위해 사용할 수 있다.
- `throw()` 메서드: 제너레이터 내부에서 처리할 예외를 호출자가 발생시킬 수 있게 만듦
- `close()` 메서드: 제너레이터 종료


## 16.2 코루틴으로 사용되는 제너레이터의 기본 동작
- 코루틴은 자신의 본체 안에 `yield` 문을 가진 일종의 제너레이터 함수로 정의된다.

In [1]:
# 예제 16-1 가장 간단한 코루틴 사용 예
def simple_coroutine():
    print('-> coroutine started')
    x = yield 3
    print('-> coroutine received: ', x)

In [2]:
my_co = simple_coroutine()
print(my_co)
print(my_co.__dir__())

<generator object simple_coroutine at 0x104e002e0>
['__repr__', '__getattribute__', '__iter__', '__next__', '__del__', 'send', 'throw', 'close', 'gi_frame', 'gi_running', 'gi_code', '__name__', '__qualname__', 'gi_yieldfrom', '__doc__', '__hash__', '__str__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__init__', '__new__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']


제너레이터가 아직 실행되지 않았으므로 `next()` 또는 `send(None)` 를 호출 (기동)해서 데이터를 전송할 수 있는 상태를 만든다.
- `next()` 전에 `send(42)`를 하면 오류가 발생한다.
- 원리는 간단하다. `send(data)`에서 data는 `yield`에 전달되는데, 아직 `yield`에 도달하지 않았기 때문이다.

In [3]:
next(my_co)
# my_co.send(None)

-> coroutine started


3

In [4]:
my_co.send(42)

-> coroutine received:  42


StopIteration: 

코루틴의 네 가지 상태
- `GEN_CREATED`: 실행을 시작하기 위해 대기하고 있는 상태
- `GEN_RUNNING`: 현재 인터프리터가 실행하고 있는 상태
- `GEN_SUSPENDED`: 현재 `yield` 문에서 값을 생성하고 이후 값이 할당될 때까지 대기하고 있는 상태
- `GEN_CLOSED`: 실행이 완료된 상태

In [5]:
# 예제 16-2 두 번 생성하는 코루틴
from inspect import getgeneratorstate


def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)
    
my_coro2 = simple_coro2(14)
print(getgeneratorstate(my_coro2))

GEN_CREATED


In [6]:
next(my_coro2)

-> Started: a = 14


14

In [7]:
print(getgeneratorstate(my_coro2))

GEN_SUSPENDED


In [8]:
my_coro2.send(28)

-> Received: b = 28


42

In [9]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [10]:
print(getgeneratorstate(my_coro2))

GEN_CLOSED


## 16.3 예제: 이동 평균을 계산하는 코루틴

- 무한 루프이므로 호출자가 값을 보내주는 한 계속해서 값을 받고 결과를 생성
- `close()` 메서드가 호출되거나, 객체에 대한 참조가 모두 사라져서 가비지 컬렉트되어야 종료된다

In [11]:
# 예제 16-3 이동 평균 코루틴 코드
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [12]:
# 예제 16-4 이동 평균 코루틴에 대한 doctest
coro_avg = averager()
next(coro_avg)

In [13]:
coro_avg.send(10)  # 다음 yield 값 생성까지 실행되므로 10을 출력한다.

10.0

In [14]:
coro_avg.send(30)

20.0

In [15]:
coro_avg.send(5)

15.0

In [16]:
coro_avg.close()

In [17]:
next(coro_avg)

StopIteration: 

## 예제 16-4 코루틴을 기동하기 위한 데커레이터

In [18]:
# 예제 16-5 코루틴을 기동하기 위한 데커레이터
from functools import wraps

def coroutine(func):
    """데커레이터: `func`를 기동해서 첫 번째 `yield`까지 진행된다."""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [19]:
# 예제 16-6 @coroutine 데커레이터를 사용한 이동 평균 코루틴의 doctest
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count
        
coro_avg = averager()
print(getgeneratorstate(coro_avg))
coro_avg.send(10)

GEN_SUSPENDED


10.0

In [20]:
coro_avg.send(30)

20.0

In [21]:
coro_avg.send(5)

15.0

## 16.5 코루틴 종료와 예외 처리
코루틴 안에서 발생한 예외를 처리해주지 않으면, 예외로 인해 코루틴이 종료되고 호출자 (`coro_avg`) 등 제너레이터 객체까지 예외가 전파된다.
호출자까지 예외가 전달되었기 때문에 더 이상 `next()`나 `send()`으로 코루틴을 다시 활성화할 수 없다.

In [22]:
# 예제 16-7 처리하지 않은 예외에 의한 코루틴 종료
coro_avg = averager()
coro_avg.send(40)
coro_avg.send(50)
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [23]:
coro_avg.send(60)

StopIteration: 

`generator.throw(exc_type[, exc_value[, traceback]])`
- 제너레이터가 중단한 곳의 `yield` 표현식에 예외를 전달
- 제너레이터에서 전달된 예외를 처리하고 있으면, 제어 흐름이 다음 `yield` 문까지 진행
- 제너레이터에서 전달된 예외를 처리하지 않으면, 호출자까지 예외가 전파

`generator.close()`
- 제너레이터가 중단한 `yield` 표현식이 `GeneratorExit` 예외를 발생시키게 만든다.
- 제너레이터가 예외를 처리하지 않거나, `StopIteration` 예외를 발생시키면 아무런 에러도 호출자에 전달되지 않는다.
- `GeneratorExit` 예외를 받으면 제너레이터는 아무런 값도 생성하지 않아야 한다. 아니면 `RuntimeError` 예외가 발생한다.

In [24]:
# 예제 16-8 코루틴의 예외 처리 방법을 설명하기 위한 제너레이터
class DemoException(Exception):
    """설명에 사용할 예외 유형"""
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print("*** DemoException handled. Continuing...")
            print(x)
        else:
            print(f"-> coroutine received: {x}")
    raise RuntimeError("This line should never run.")

In [25]:
# 예제 16-9 예외를 발생시키지 않는 demo_exc_handling()의 활성화 및 종료
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.send(22)
exc_coro.close()
getgeneratorstate(exc_coro)

-> coroutine started
-> coroutine received: 11
-> coroutine received: 22


'GEN_CLOSED'

In [26]:
# 예제 16-10 DemoException을 demo_exc_handling() 안에 던져도 종료되지 않는다.
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.throw(DemoException)
exc_coro.send(11)

-> coroutine started
-> coroutine received: 11
*** DemoException handled. Continuing...
11
-> coroutine received: 11


In [27]:
# 예제 16-11 자신에게 던져진 예외를 처리할 수 없으면 코루틴이 종료된다.
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.throw(Exception)

-> coroutine started


Exception: 

In [28]:
exc_coro.send(11)

StopIteration: 

코루틴이 어떻게 종료되든 무조건 실행해야 할 코드 (close 등 정리 코드)를 실행해야 하는 경우에는 `finally` 구문을 사용한다.

In [29]:
# 예제 16-12
def demo_finally():
    print('-> Coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print("*** DemoException handled. Continuing...")
            else:
                print(f'-> Coroutine received: {x}')
    finally:
        print("-> Coroutine ending")

## 16-6 코루틴에서 값 반환하기

In [30]:
# 16-13 결과를 반환하는 average() 코루틴 코드
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
        print(f'Current average: {average}')
    return Result(count, average)

In [31]:
# 예제 16-14 averager()의 동작을 보여주는 예제
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

Current average: 10.0
Current average: 20.0
Current average: 15.5


StopIteration: Result(count=3, average=15.5)

In [32]:
# 예제 16-15 StopIteration을 잡으면 averager()가 반환하는 값을 가져올 수 있다.
# 예제 16-14 averager()의 동작을 보여주는 예제
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
result

Current average: 10.0
Current average: 20.0
Current average: 15.5


Result(count=3, average=15.5)

## 16.7 `yield from` 사용하기
- `yield from`은 `yield`와 다른 완전히 새로운 언어 구성체이며 `yield`보다 더 많은 일을 한다. 다른 언어에서 `await`와 유사하다.

`yield from` 구성체는 `StopIteration` 예외를 내부적으로 잡아서 자동으로 처리한다.
- `for` 루프가 제너레이터가 모두 소진되었을 때 `StopIteration`을 처리하는 것과 유사하다.
- 예외의 `value` 속성 (`exc.value`)을 반환한다.

- `gen()`이 `yield from subegn()`을 호출하고, `subgen()`이 이어받아 값을 생성하고 `gen()`의 호출자에 반환한다.
  - 실질적으로 `subgen()`이 직접 호출자를 이끈다. 그러는 동안 `gen()`은 `subgen()`이 종료될 떄까지 실행을 중단한다.

In [33]:
# 예제 16-16 yield from으로 반복형 객체를 연결하기
def chain(*iterables):
    for it in iterables:
        yield from it
        
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

- `yield from x` 표현식이 `x` 객체에 대해 첫 번째로 하는 일은 `iter(x)`를 호출해서 `x`의 반복자를 가져오는 것이다.
- `yield from`의 가장 바깥쪽 호출자와 가장 안쪽에 있는 하위 제너레이터 사이에 양방향 채널을 열어준다는 것이다.
- 모든 `yield from` 체인은 가장 바깥쪽 대표 제너레이터에 `next()`와 `send()`를 호출하는 클라이언트에 의해 주도된다.

<br>

- 대표 제너레이터: `yield from <반복형>` 표현식을 담고 있는 제너레이터 함수
- 하위 제너레이터: `yield from <반복형>` 중 <반복형>에서 가져오는 제너레이터
- 호출자: 대표제너레이터를 호출하는 코드

In [34]:
# 예제 16-17 yield from을 이용해서 averager()를 구동하고 보고서 생성하기
from collections import namedtuple

Result = namedtuple('Result', 'count average')

data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}


# 대표 제너레이터
def grouper(results, key):
    while True:
        results[key] = yield from averager()

# 하위 제너레이터
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)
    
    
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print(f"{result.count:2}{group:5} average {result.average:.2f}{unit}")


# 호출자
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    report(results)
    
main(data)

 9boys  average 40.42kg
 9boys  average 1.39m
10girls average 42.04kg
10girls average 1.43m


~~~python
results = {}
for key, values in data.items():
    group = grouper(results, key)
    next(group)
~~~

-> `group` 변수에 `grouper()` 제너레이터 객체를 할당하고 `next(group)`을 통해 기동

<br>

~~~python
while True:
    results[key] = yield from averager()
~~~
-> 하위 제너레이터 `averager()`를 호출 후 `yield from`에서 대기

<br>

~~~python
results = {}
for key, values in data.items():
    group = grouper(results, key)
    next(group)
    for value in values:
        group.send(value)
    group(None)
~~~
-> `value`가 하위 제너레이터인 `averager()`의 `yield` 문에 값이 전달된다...!<br>
-> `values` 리스트의 값들을 다 전송 후 `None`을 전송하여 `averager()` 제너레이터가 `return` 문까지 진행시키고, 이를 `results[key]`에 저장한다.

## 16.8 `yield from`의 의미
- 클라이언트 (호출자, 주로 main() 함수)가 `next()`와 `send()` 뿐만 아니라 `throw()`와 `close()`를 호출할 수도 있다.
  - 만약 하위 제너레이터가 `throw()`와 `close()` 메서드가 없는 단순한 반복형 경우일 경우 `yield from` 논리에서 처리해줘야 한다.

책 참조.

## 16.9 사용 사례: 이산 이벤트 시뮬레이션을 위한 코루틴
### 16.9.1 이산 이벤트 시뮬레이션
- **연속 시뮬레이션**
  - 단위 시간마다 시스템의 상태가 기록된다. 
  - 단위 시간만큼 시간이 흐르면서 이벤트가 발생하고 시스템의 상태가 바뀌게 된다.
  - 다중 스레드를 이용해서 구현
- **이산 이벤트 시뮬리에션**
  - 이벤트 단위마다 시스템의 상태가 기록된다.
  - 코루틴을 사용해서 구현 

### 16.9.2 택시 집단 시뮬레이션
이벤트
- Leaving garage
- Picking up passenger
- Drop off passenger
- Going home

디테일
- 승객을 태우기까지의 시간과 운행 시간은 지수 분포를 이용해서 생성 
- 각 택시는 앞 차가 출발한지 5분후에 차고를 출발한다.


In [35]:
import collections

Event = collections.namedtuple('Event', 'time proc action')

`taxi_process()`: 각 택시마다 한 번씩 호출되어 택시의 행동을 나타내는 제너레이터

In [36]:
# 예제 16-20 각 택시의 행동을 구현하는 taxi_process() 코루틴
def taxi_process(ident, trips, start_time=0):
    """각 단계 변화마다 이벤트를 생성하며 시뮬레이터에 제어권을 넘긴다."""
    time = yield Event(start_time, ident, 'leave garage')  # 출발 이벤트를 넘겨주고, 탑승 이벤트 발생 시간을 넘겨 받는다.
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')  # 탑승 이벤트를 넘겨주고, 하차 이벤트 발생 시간을 넘겨 받는다.
        time = yield Event(time, ident, 'drop off passenger')  # 하차 이벤트를 넘겨주고, 탑승 또는 귀가 이벤트 발생 시간을 넘겨 받는다.
    yield Event(time, ident, 'going home')

In [37]:
# 예제 16-21 taxi_process() 코루틴 돌려보기
taxi = taxi_process(ident=13, trips=2, start_time=0)
next(taxi)

Event(time=0, proc=13, action='leave garage')

In [38]:
taxi.send(_.time + 7)

Event(time=7, proc=13, action='pick up passenger')

In [39]:
taxi.send(_.time + 23)

Event(time=30, proc=13, action='drop off passenger')

In [40]:
taxi.send(_.time + 5)

Event(time=35, proc=13, action='pick up passenger')

In [41]:
taxi.send(_.time + 48)

Event(time=83, proc=13, action='drop off passenger')

In [42]:
taxi.send(_.time + 5)

Event(time=88, proc=13, action='going home')

In [43]:
taxi.send(_.time + 10)

StopIteration: 

In [44]:
# 예제 16-23 기본 뼈대를 갖춘 Simulator 클래스
import queue
import random


DEPARTURE_INTERVAL = 5
SEARCH_DURATION = 5
TRIP_DURATION = 20
DEFAULT_END_TIME = 180
DEFAULT_NUMBER_OF_TAXIS = 3


def compute_duration(previous_action):
    if previous_action in ['leave garage', 'drop off passenger']:
        interval = SEARCH_DURATION
    elif previous_action == 'pick up passenger':
        interval = TRIP_DURATION
    elif previous_action == 'going home':
        interval = 1
    else:
        raise ValueError(f"Unknown previous action: {previous_action}")
    return int(random.expovariate(1/interval)) + 1
    

class Simulator:
    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)
    
    def run(self, end_time):
        # 각 택시의 첫 번째 이벤트 스케줄링
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)
        
        # 시뮬레이션 핵심 루프
        sim_time = 0
        while sim_time < end_time:
            if self.events.empty():
                print("*** end of events ***")
                break
            
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event
            print("Taxi: ", proc_id, proc_id * '   ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]  # 귀가 후에 send()시 발생하니깐 지워주기
            else:
                self.events.put(next_event)
        else:
            msg = "*** end of simulation time: {} events pending"
            print(msg.format(self.events.qsize()))    


random.seed(3)
taxis = {i: taxi_process(i, (i + 1) * 2, i * DEPARTURE_INTERVAL) 
         for i in range(DEFAULT_NUMBER_OF_TAXIS)}
sim = Simulator(taxis)
sim.run(DEFAULT_END_TIME)
    

Taxi:  0  Event(time=0, proc=0, action='leave garage')
Taxi:  0  Event(time=2, proc=0, action='pick up passenger')
Taxi:  1     Event(time=5, proc=1, action='leave garage')
Taxi:  1     Event(time=8, proc=1, action='pick up passenger')
Taxi:  2        Event(time=10, proc=2, action='leave garage')
Taxi:  2        Event(time=15, proc=2, action='pick up passenger')
Taxi:  2        Event(time=17, proc=2, action='drop off passenger')
Taxi:  0  Event(time=18, proc=0, action='drop off passenger')
Taxi:  2        Event(time=18, proc=2, action='pick up passenger')
Taxi:  2        Event(time=25, proc=2, action='drop off passenger')
Taxi:  1     Event(time=27, proc=1, action='drop off passenger')
Taxi:  2        Event(time=27, proc=2, action='pick up passenger')
Taxi:  0  Event(time=28, proc=0, action='pick up passenger')
Taxi:  2        Event(time=40, proc=2, action='drop off passenger')
Taxi:  2        Event(time=44, proc=2, action='pick up passenger')
Taxi:  1     Event(time=55, proc=1, action

In [45]:
# 하위 제너레이터. 사용자에게 받은 데이터를 받아서 제곱을 출력함
# 접근할 수 없는 private 함수라고 가정
def _writer():
    """A coroutine that writes data *sent* to it to fd, socket, etc."""
    while True:
        w = (yield)
        print('>> ', w ** 2)
        
gen = _writer()
next(gen)
gen.send(10)

>>  100


In [46]:
# 하위 제너레이터. 사용자에게 받은 데이터를 받아서 제곱을 출력함
# 접근할 수 없는 private 함수라고 가정
def _writer():
    """A coroutine that writes data *sent* to it to fd, socket, etc."""
    while True:
        w = (yield)
        print('>> ', w ** 2)

# 대표 제너레이터. 사용자에게 데이터를 받아서, _writer에게 보냄
# 접근할 수 있는 공개 함수
def writer_wrapper():
    gen = _writer()
    next(gen)
    while True:
        x = (yield)
        gen.send(x)

# 호출자
gen = writer_wrapper()
next(gen)
gen.send( 10 )  # read_data를 통해, _get_data()에 10을 보내서 10 ** 2를 받고 싶음

>>  100


In [47]:
gen.send(3)

>>  9


In [48]:
gen.send(9)

>>  81


In [49]:
class SpamException(Exception):
    pass

def _writer():
    while True:
        try:
            w = (yield)
        except SpamException:
            print('***')
        else:
            print('>> ', w)


def writer_wrapper():
    gen = _writer()
    next(gen)
    while True:
        x = (yield)
        gen.send(x)

wrap = writer_wrapper()
wrap.send(None)  # "prime" the coroutine
for i in [0, 1, 2, 'spam', 4]:
    if i == 'spam':
        wrap.throw(SpamException)
    else:
        wrap.send(i)

>>  0
>>  1
>>  2


SpamException: 

In [50]:
class SpamException(Exception):
    pass

def _writer():
    while True:
        try:
            w = (yield)
        except SpamException:
            print('***')
        else:
            print('>> ', w)


def writer_wrapper():
    gen = _writer()
    next(gen)
    while True:
        try:
            x = (yield)
        except Exception as e:
            gen.throw(e)
        else:
            gen.send(x)


wrap = writer_wrapper()
wrap.send(None)  # "prime" the coroutine
for i in [0, 1, 2, 'spam', 4]:
    if i == 'spam':
        wrap.throw(SpamException)
    else:
        wrap.send(i)

>>  0
>>  1
>>  2
***
>>  4


In [51]:
class SpamException(Exception):
    pass

def _writer():
    while True:
        try:
            w = (yield)
        except SpamException:
            print('***')
        else:
            print('>> ', w)


def writer_wrapper():
    yield from _writer()


wrap = writer_wrapper()
wrap.send(None)  # "prime" the coroutine
for i in [0, 1, 2, 'spam', 4]:
    if i == 'spam':
        wrap.throw(SpamException)
    else:
        wrap.send(i)

>>  0
>>  1
>>  2
***
>>  4
